In [1]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [2]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [3]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [4]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

        DROP TABLE IF EXISTS neo;

        CREATE TABLE neo (
                        id text,
                        data json
        )
    ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                        (neo['id'], json.dumps(neo)))
    conn.commit()

Exploration of SQL transformation

In [5]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [6]:
df = pd.read_sql_query('''SELECT data FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2137084,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2137084?ap...,1.338305,2.992540,1338.304618,2992.540101,0.831584,1.859478,4390.763323,9818.045264
1,2252399,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2252399?ap...,0.342417,0.765667,342.416731,765.667087,0.212768,0.475763,1123.414507,2512.031205
2,3342645,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3342645?ap...,0.072536,0.162196,72.536229,162.195940,0.045072,0.100784,237.979763,532.138928
3,3451020,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3451020?ap...,0.145397,0.325117,145.396843,325.117225,0.090345,0.202018,477.023778,1066.657595
4,3606252,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3606252?ap...,0.197040,0.440595,197.040262,440.595420,0.122435,0.273773,646.457574,1445.523079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,54313606,54313606,(2022 TB1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313606,25.15,False,"[{'close_approach_date': '2022-10-06', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54313606?a...,0.024806,0.055468,24.805899,55.467677,0.015414,0.034466,81.384187,181.980574
6342,54313607,54313607,(2022 TC1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313607,24.56,False,"[{'close_approach_date': '2022-10-06', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54313607?a...,0.032550,0.072785,32.550299,72.784680,0.020226,0.045226,106.792322,238.794891
6343,54317176,54317176,(2022 UB2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54317176,22.79,False,"[{'close_approach_date': '2022-10-06', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54317176?a...,0.073545,0.164452,73.545309,164.452310,0.045699,0.102186,241.290391,539.541717
6344,54320199,54320199,(2022 UP9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54320199,22.05,False,"[{'close_approach_date': '2022-10-06', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54320199?a...,0.103408,0.231228,103.408200,231.227764,0.064255,0.143678,339.265757,758.621296


Extracting nested close approach data

In [7]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2023-10-01',
  'close_approach_date_full': '2023-Oct-01 17:21',
  'epoch_date_close_approach': 1696180860000,
  'relative_velocity': {'kilometers_per_second': '18.0203381913',
   'kilometers_per_hour': '64873.2174887045',
   'miles_per_hour': '40309.6945001342'},
  'miss_distance': {'astronomical': '0.4544143686',
   'lunar': '176.7671893854',
   'kilometers': '67979421.639954882',
   'miles': '42240453.8911940916'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [8]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_10112\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}
C:\Users\Jack\AppData\Local\Temp\ipykernel_10112\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}
C:\Users\Jack\AppData\Local\Temp\ipykernel_10112\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

C:\Users\Jack\AppData\Local\Temp\ipykernel_10112\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}
C:\Users\Jack\AppData\Local\Temp\ipykernel_10112\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}
C:\Users\Jack\AppData\Local\Temp\ipykernel_10112\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2137084,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2137084?ap...,1.338305,2.992540,1338.304618,2992.540101,0.831584,1.859478,4390.763323,9818.045264
1,2252399,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2252399?ap...,0.342417,0.765667,342.416731,765.667087,0.212768,0.475763,1123.414507,2512.031205
2,3342645,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3342645?ap...,0.072536,0.162196,72.536229,162.195940,0.045072,0.100784,237.979763,532.138928
3,3451020,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3451020?ap...,0.145397,0.325117,145.396843,325.117225,0.090345,0.202018,477.023778,1066.657595
4,3606252,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3606252?ap...,0.197040,0.440595,197.040262,440.595420,0.122435,0.273773,646.457574,1445.523079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,54313606,54313606,(2022 TB1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313606,25.15,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54313606?a...,0.024806,0.055468,24.805899,55.467677,0.015414,0.034466,81.384187,181.980574
6342,54313607,54313607,(2022 TC1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313607,24.56,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54313607?a...,0.032550,0.072785,32.550299,72.784680,0.020226,0.045226,106.792322,238.794891
6343,54317176,54317176,(2022 UB2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54317176,22.79,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54317176?a...,0.073545,0.164452,73.545309,164.452310,0.045699,0.102186,241.290391,539.541717
6344,54320199,54320199,(2022 UP9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54320199,22.05,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54320199?a...,0.103408,0.231228,103.408200,231.227764,0.064255,0.143678,339.265757,758.621296


In [9]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2023-10-01,2023-Oct-01 17:21,1696180860000,Earth,18.0203381913,64873.2174887045,40309.6945001342,0.4544143686,176.7671893854,67979421.639954882,42240453.8911940916
1,2023-10-01,2023-Oct-01 23:55,1696204500000,Earth,13.1737562484,47425.5224941968,29468.3753520191,0.4153101202,161.5556367578,62129509.371363974,38605486.9631478812
2,2023-10-01,2023-Oct-01 10:30,1696156200000,Earth,11.207421835,40346.7186060047,25069.8819027144,0.3264809883,127.0011044487,48840860.445174921,30348303.4111030698
3,2023-10-01,2023-Oct-01 06:34,1696142040000,Earth,19.7494474735,71098.0109047015,44177.5390535352,0.1343616471,52.2666807219,20100216.215851677,12489695.2016680626
4,2023-10-01,2023-Oct-01 23:20,1696202400000,Earth,9.4421483153,33991.7339350502,21121.1415664761,0.1685102646,65.5504929294,25208776.657296402,15664007.4651662676
...,...,...,...,...,...,...,...,...,...,...,...
6341,2022-10-06,2022-Oct-06 05:20,1665033600000,Earth,7.5262324394,27094.4367817928,16835.4293436646,0.0294355933,11.4504457937,4403502.059866271,2736209.3002896998
6342,2022-10-06,2022-Oct-06 09:22,1665048120000,Earth,22.6026467313,81369.5282325724,50559.8604731781,0.0317376154,12.3459323906,4747879.662719198,2950195.6200250124
6343,2022-10-06,2022-Oct-06 10:56,1665053760000,Earth,9.4720235913,34099.2849285446,21187.9695712948,0.0952070391,37.0355382099,14242770.258366717,8850047.0564140146
6344,2022-10-06,2022-Oct-06 17:18,1665076680000,Earth,12.8432296399,46235.6267035576,28729.0203814411,0.1804917856,70.2113045984,27001186.678256672,16777759.4068271936


In [10]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,data
0,2137084,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2137084?ap...,1.338305,...,1696180860000,Earth,18.0203381913,64873.2174887045,40309.6945001342,0.4544143686,176.7671893854,67979421.639954882,42240453.8911940916,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2252399,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2252399?ap...,0.342417,...,1696204500000,Earth,13.1737562484,47425.5224941968,29468.3753520191,0.4153101202,161.5556367578,62129509.371363974,38605486.9631478812,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3342645,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3342645?ap...,0.072536,...,1696156200000,Earth,11.207421835,40346.7186060047,25069.8819027144,0.3264809883,127.0011044487,48840860.445174921,30348303.4111030698,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3451020,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3451020?ap...,0.145397,...,1696142040000,Earth,19.7494474735,71098.0109047015,44177.5390535352,0.1343616471,52.2666807219,20100216.215851677,12489695.2016680626,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3606252,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3606252?ap...,0.197040,...,1696202400000,Earth,9.4421483153,33991.7339350502,21121.1415664761,0.1685102646,65.5504929294,25208776.657296402,15664007.4651662676,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,54313606,54313606,(2022 TB1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313606,25.15,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54313606?a...,0.024806,...,1665033600000,Earth,7.5262324394,27094.4367817928,16835.4293436646,0.0294355933,11.4504457937,4403502.059866271,2736209.3002896998,{'links': {'self': 'http://api.nasa.gov/neo/re...
6342,54313607,54313607,(2022 TC1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313607,24.56,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54313607?a...,0.032550,...,1665048120000,Earth,22.6026467313,81369.5282325724,50559.8604731781,0.0317376154,12.3459323906,4747879.662719198,2950195.6200250124,{'links': {'self': 'http://api.nasa.gov/neo/re...
6343,54317176,54317176,(2022 UB2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54317176,22.79,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54317176?a...,0.073545,...,1665053760000,Earth,9.4720235913,34099.2849285446,21187.9695712948,0.0952070391,37.0355382099,14242770.258366717,8850047.0564140146,{'links': {'self': 'http://api.nasa.gov/neo/re...
6344,54320199,54320199,(2022 UP9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54320199,22.05,False,"{'close_approach_date': '2022-10-06', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54320199?a...,0.103408,...,1665076680000,Earth,12.8432296399,46235.6267035576,28729.0203814411,0.1804917856,70.211304

Cleaning up df

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6346 entries, 0 to 6345
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    6346 non-null   object 
 1   neo_reference_id                                      6346 non-null   object 
 2   name                                                  6346 non-null   object 
 3   nasa_jpl_url                                          6346 non-null   object 
 4   absolute_magnitude_h                                  6346 non-null   float64
 5   is_potentially_hazardous_asteroid                     6346 non-null   bool   
 6   close_approach_data                                   6346 non-null   object 
 7   is_sentry_object                                      6346 non-null   bool   
 8   links.self                                            6346

In [12]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,data
0,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,False,1338.304618,2992.540101,2023-Oct-01 17:21,Earth,18.0203381913,64873.2174887045,0.4544143686,176.7671893854,67979421.639954882,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,False,342.416731,765.667087,2023-Oct-01 23:55,Earth,13.1737562484,47425.5224941968,0.4153101202,161.5556367578,62129509.371363974,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,False,72.536229,162.195940,2023-Oct-01 10:30,Earth,11.207421835,40346.7186060047,0.3264809883,127.0011044487,48840860.445174921,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,False,145.396843,325.117225,2023-Oct-01 06:34,Earth,19.7494474735,71098.0109047015,0.1343616471,52.2666807219,20100216.215851677,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,False,197.040262,440.595420,2023-Oct-01 23:20,Earth,9.4421483153,33991.7339350502,0.1685102646,65.5504929294,25208776.657296402,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,54313606,(2022 TB1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313606,25.15,False,False,24.805899,55.467677,2022-Oct-06 05:20,Earth,7.5262324394,27094.4367817928,0.0294355933,11.4504457937,4403502.059866271,{'links': {'self': 'http://api.nasa.gov/neo/re...
6342,54313607,(2022 TC1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313607,24.56,False,False,32.550299,72.784680,2022-Oct-06 09:22,Earth,22.6026467313,81369.5282325724,0.0317376154,12.3459323906,4747879.662719198,{'links': {'self': 'http://api.nasa.gov/neo/re...
6343,54317176,(2022 UB2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54317176,22.79,False,False,73.545309,164.452310,2022-Oct-06 10:56,Earth,9.4720235913,34099.2849285446,0.0952070391,37.0355382099,14242770.258366717,{'links': {'self': 'http://api.nasa.gov/neo/re...
6344,54320199,(2022 UP9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54320199,22.05,False,False,103.408200,231.227764,2022-Oct-06 17:18,Earth,12.8432296399,46235.6267035576,0.1804917856,70.2113045984,27001186.678256672,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [13]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter.meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter.meters_max'
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters_min,estimated_diameter.meters_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,data
0,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,False,1338.304618,2992.540101,2023-Oct-01 17:21,Earth,18.0203381913,64873.2174887045,0.4544143686,176.7671893854,67979421.639954882,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,False,342.416731,765.667087,2023-Oct-01 23:55,Earth,13.1737562484,47425.5224941968,0.4153101202,161.5556367578,62129509.371363974,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,False,72.536229,162.195940,2023-Oct-01 10:30,Earth,11.207421835,40346.7186060047,0.3264809883,127.0011044487,48840860.445174921,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,False,145.396843,325.117225,2023-Oct-01 06:34,Earth,19.7494474735,71098.0109047015,0.1343616471,52.2666807219,20100216.215851677,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,False,197.040262,440.595420,2023-Oct-01 23:20,Earth,9.4421483153,33991.7339350502,0.1685102646,65.5504929294,25208776.657296402,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,54313606,(2022 TB1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313606,25.15,False,False,24.805899,55.467677,2022-Oct-06 05:20,Earth,7.5262324394,27094.4367817928,0.0294355933,11.4504457937,4403502.059866271,{'links': {'self': 'http://api.nasa.gov/neo/re...
6342,54313607,(2022 TC1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54313607,24.56,False,False,32.550299,72.784680,2022-Oct-06 09:22,Earth,22.6026467313,81369.5282325724,0.0317376154,12.3459323906,4747879.662719198,{'links': {'self': 'http://api.nasa.gov/neo/re...
6343,54317176,(2022 UB2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54317176,22.79,False,False,73.545309,164.452310,2022-Oct-06 10:56,Earth,9.4720235913,34099.2849285446,0.0952070391,37.0355382099,14242770.258366717,{'links': {'self': 'http://api.nasa.gov/neo/re...
6344,54320199,(2022 UP9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54320199,22.05,False,False,103.408200,231.227764,2022-Oct-06 17:18,Earth,12.8432296399,46235.6267035576,0.1804917856,70.2113045984,27001186.678256672,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [14]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2022, 10, 3)

In [15]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2023, 10, 2)

In [16]:
delta = (datetime.date.today() - last_execution_date).days
delta

1

In [17]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [84]:
delta = 8

for x in range (0, delta, 7):
    print(x)

0
7


In [93]:
delta = 14
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta == 7:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)
    else:
        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    return links

get_links() 
        

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-26&end_date=2023-10-02&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-19&end_date=2023-09-25&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [106]:
delta = 7
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    if delta % 7 == 0:
        # end_date = datetime.date.today()
        # start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        # end_date = end_date.strftime('%Y-%m-%d')
        # link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        # links.append(link)

        for x in range(0, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)  
        
    return links

get_links()

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-25&end_date=2023-10-02&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [90]:
delta = 14
def get_links():
    if delta == 0:
        return []

    links = []

    def build_link(x, y):
        end_date = x
        start_date = (end_date - datetime.timedelta(days=y)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta % 7 != 0:
        build_link(datetime.date.today(), delta%7)
    

    if delta == 7:
        build_link(datetime.date.today(), 7)

    else:
        for x in range(delta%7, delta, 7):
            build_link((datetime.date.today() - datetime.timedelta(days=x+1)), 6)

    return links

get_links() 

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-26&end_date=2023-10-02&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-19&end_date=2023-09-25&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [21]:
def load_data():
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            id text,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (neo['id'], json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

In [22]:
from main import transform

load_data(), transform()

c:\Users\Jack\Desktop\Coding\NASA\main.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


            id               name  \
0      2164207  164207 (2004 GU9)   
1      2504800  504800 (2010 CO1)   
2      3401428         (2008 CL1)   
3      3748455        (2016 FA13)   
4      3773752         (2017 GK4)   
...        ...                ...   
7711   3843536         (2019 PM2)   
7712   3989291        (2020 BD13)   
7713  54051300         (2020 QG3)   
7714  54115824         (2021 CN5)   
7715  54336908         (2022 YL5)   

                                        nasa_jpl_url  absolute_magnitude_h  \
0      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207                 21.19   
1      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800                 21.48   
2      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428                 19.49   
3      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455                 22.50   
4      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752                 21.79   
...                                              ...                   ...   
7711   ht

(None, None)